# Description
This notebook is filtering the results based on the information given in `.xlsx` file. 

`.xlsx` file contains two sheets:
- `TMC` with list of all tmcs
- `Time` with `Start Time` and `End Time` columns

The code is working as follows:
- first it prepares a file `TMC_FILE` that contains all the records with TMCs from the list
- secondly it prepares a `FINAL_FILE` that contains the TMCs filtered according to the excel

In [ ]:
import pandas as pd
import numpy as np
from time import time, strftime, gmtime
import sys

In [ ]:
TMC_FILE = './selected_tmcs.csv' # File with selected tmcs
FINAL_FILE = './filtered_tmcs.csv' # Final file, with time-based filter

SOURCE_FILE = '/hdd3/Volumes/MD2018/estimates/estimates_{}.csv' # Path to file(s) with estimations

# Load Excel with data

In [ ]:
xls = pd.ExcelFile('TMCListSara_20200416.xlsx')
print (xls.sheet_names)
tmc_df = pd.read_excel(xls, 'TMC')
fromTo = pd.read_excel(xls, 'Time')
tmc_df.head(3)

In [ ]:
print ('Start time - min: {}, max: {}.'.format(fromTo['Start Time'].min(), fromTo['Start Time'].max()))
print ('End time - min: {}, max: {}.'.format(fromTo['End Time'].min(), fromTo['End Time'].max()))

first_month = int(fromTo['Start Time'].min().split('-')[1])
last_month = int(fromTo['End Time'].max().split('-')[1])
print ('First month: {}, last_month: {}'.format(first_month, last_month))

fromTo.head(3)

In [ ]:
tmc_list = list(tmc_df.tmc)
print (tmc_list)

# Filter tmc lists

In [ ]:
if True: # Prepare the file that contains all the data for selected TMCs
    lines_all = 0
    lines_saved = 0
    t = time()
    save_header = True
    
    with open(TMC_FILE, 'w') as dest:
        for month in np.arange(first_month, last_month+1):
            filename = SOURCE_FILE.format(str(month).zfill(2))
            with open(filename, 'r') as source:
                print ('Start reading from {} ({}).'.format(filename, strftime('%H:%M:%S', gmtime(time() - t))))
                line = source.readline()
                if save_header:
                    dest.write(line)
                    save_header = False

                while True:
                    line = source.readline()
                    if not line:
                        break
                    tmc = line.split(',')[0]
                    if tmc in tmc_list:
                        dest.write(line)
                        lines_saved += 1

                    lines_all += 1
                    if lines_all % 3e5 == 0:
                        sys.stdout.write('{:,} lines processed, {:,} ({:.1f}%) lines saved.    \r'.format(lines_all, lines_saved, 100*lines_saved / lines_all))        

    print ('Done. {:,} lines saved in {}s. {:,} lines processed ({:,} lines/s)'.format(
        lines_saved, strftime('%H:%M:%S', gmtime(time() - t)), lines_all, int(lines_all / (time() - t))))

# Filter by time

In [ ]:
df = pd.read_csv(TMC_FILE)
df['datetime'] = pd.to_datetime(df.date + ' ' + df.time, utc=None) 
df.head()

In [ ]:
fromTo['start'] = pd.to_datetime(fromTo['Start Time'].apply(lambda x: x[:-6]), utc=None)
fromTo['end'] = pd.to_datetime(fromTo['End Time'].apply(lambda x: x[:-6]), utc=None)

In [ ]:
for idx, row in fromTo.iterrows():
    sys.stdout.write('Processing date {}/{} ({:.1f}% done).    \r'.format(idx+1, len(fromTo), 100*idx/len(fromTo)))
    colname = 'TimeRow_{}'.format(idx)
    df[colname] = (df.datetime >= row.start) & (df.datetime <= row.end)
    df[colname] = df[colname].astype(int)

added_cols = [x for x in df.columns if x[:8] == 'TimeRow_']
df['inDates'] = df[added_cols].sum(axis='columns')
df = df.drop(added_cols, axis='columns')

print ('\nDone.')


In [ ]:
print ('Original shape:', df.shape)
df = df[df.inDates >= 1]
print ('Shape after filtering:', df.shape)
df.set_index('tmc').to_csv(FINAL_FILE)